In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Данный ноутбук представлен в виде видео презентации по ссылке: https://cloud.mail.ru/public/CYZ8/Dkvfdt6kP, где содержится более подробное повествование

# Data pre-processing

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_oct = pd.read_csv('../input/ecommerce-events-history-in-cosmetics-shop/2019-Oct.csv')
df_nov = pd.read_csv('../input/ecommerce-events-history-in-cosmetics-shop/2019-Nov.csv')
df_dec = pd.read_csv('../input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv')
df_feb = pd.read_csv('../input/ecommerce-events-history-in-cosmetics-shop/2020-Feb.csv')
df_jan = pd.read_csv('../input/ecommerce-events-history-in-cosmetics-shop/2020-Jan.csv')

In [ ]:
df = df_oct.append([df_nov, df_dec, df_feb, df_jan])

In [ ]:
df.isnull().sum() / len(df) # оценим долю пропусков

In [ ]:
df.drop_duplicates(inplace = True) # удалю повторяющиеся строки
df.drop(['category_code', 'product_id', 'category_id', 'brand'], axis = 1, inplace = True) # удалю неиспользовавшиеся столбцы
df.loc[(df.price < 0), 'price'] = abs(df.loc[(df.price < 0), 'price']) # избавлюсь от отрицательных цен
df.event_time = pd.to_datetime(df.event_time.str[0:19]) # удалю 'UTC'

In [ ]:
df.iloc[307:313]

# Retention by months

In [ ]:
start = df.groupby('user_id')['event_time'].min() 
df = df.merge(start, on = 'user_id').rename(columns = {'event_time_x': 'event_time', 'event_time_y': 'start_date'})
df.sort_values('event_time', inplace = True)

df['N_month'] = np.ceil((df['event_time'] - df['start_date']).dt.days  / 30) # за месяц считается период в 30 дней
df.N_month = df.N_month.astype('int64')

df.loc[df.N_month == 6, 'N_month'] = 5  # т.к. данные собраны за 152 дня, 2 дня остаются лишними, присвою их к 5-ому месяцу
df.head() # N_month = 0 - день первого взаимодействия с магазином

In [ ]:
monthly_ret = df.groupby('N_month')['user_id'].nunique() / df.user_id.nunique() * 100
plt.figure(figsize = (16, 9))
g = sns.barplot(x = monthly_ret.index, y = monthly_ret.values, color = '#654321')

plt.title('Retention by months', fontsize= 20)
g.set_yticklabels([str(y) + '%' for y in range(0, 120, 20)], fontsize = 15)
g.set_xticklabels(df.N_month.unique() ,fontsize = 15)
plt.xlabel('')

plt.show()

In [ ]:
from IPython import display
display.Image('../input/picture/Consumer frequency.jpg')

# Когортный анализ покупателей, совершающих покупки через N месяцев после первой активности

Выделим когорты по неделе старта в магазине

In [ ]:
for week in df.start_date.dt.isocalendar().week.unique():
    df.loc[df.start_date.dt.isocalendar().week == week, 'start_week'] = df.loc[df.start_date.dt.isocalendar().week == week, 'start_date'].dt.date.min()

Вывести события только для покупателей

In [ ]:
purchasers = pd.Series(df.loc[df.event_type == 'purchase', 'user_id'].unique()).rename('user_id')
purchasers_events = df.merge(purchasers)

Доля пользователей совершающих покупки на N-ый месяц 'жизни'

In [ ]:
# 0 - любая активность в день привлечения этого покупателя.
df_purchases = purchasers_events.loc[purchasers_events.N_month == 0].append(purchasers_events.loc[(purchasers_events.event_type == 'purchase') & (purchasers_events.N_month > 0)])

cohorts = (df_purchases.groupby(['start_week', 'N_month'])['user_id'].nunique() / df_purchases.groupby('start_week')['user_id'].nunique()).reset_index()
purchases_by_users = cohorts.pivot_table(index = 'start_week', columns = 'N_month', values = 'user_id')

plt.figure(figsize = (16, 9))
k = sns.heatmap(purchases_by_users, annot = True, fmt = '.0%', vmin = 0.0, vmax = 0.5, cmap="YlGnBu")

plt.title('Monthly purchases by cohorts', fontsize = 20)
plt.ylabel('start week', fontsize = 17)
plt.xlabel('')

k.set_yticklabels(df_purchases.start_week.unique() ,fontsize = 13)
k.set_xticklabels(cohorts.N_month.unique(),fontsize = 15)

plt.show()

In [ ]:
a = (df.event_time.max().date() - df.start_week.unique())
a = pd.Series(a).dt.days

mean_dict = {}
for i in range(6):
    mean_dict[i] = purchases_by_users[i][a[a >= 30 * i].index].mean() # Средние значения только для когорт, успевших "прожить" данный месяц

In [ ]:
plt.figure(figsize = (16, 5))
m = sns.barplot(x = list(mean_dict.keys()), y = list(mean_dict.values()), color = '#654321')

plt.title('Mean_values', fontsize= 20)
m.set_yticklabels([str(y) + '%' for y in range(0, 120, 20)], fontsize = 15)
m.set_xticklabels(m.get_xticks() ,fontsize = 15)
plt.xlabel('')
plt.axhline(0.66, color = 'red')

plt.show()

In [ ]:
delta = (0.66 - pd.Series(mean_dict)[1:]) # разница средних и 0.66 
delta

In [ ]:
b = []
for i in range(6):
    b.append(df_purchases.groupby('start_week')['user_id'].nunique()[a[a >= 30 * i].index].sum()) 
b = pd.Series(b)

In [ ]:
b.shift(+1) * delta * 0.35 * 29 * 0.7 # увеличение оборота на каждом из месяцев 'жизни' покупателя

In [ ]:
(b.shift(+1) * delta * 0.35 * 29 * 0.7).sum() # суммарный прирост

При оценке эффекта в презентации приводится пример с бонусами, при этом, исходя из исследования, в retail'е 35% пользователей дают доступ к push-уведомлениям. Однако бонусы и иные выгодные предложения рассылаются также через СМС и почту. Доля в 35% взята как ориентир, чтобы оценить, какая часть пользователей ознакомится с предложением

In [ ]:
from IPython import display
display.Image('../input/picture-2/vmXoBlnlxQ4.jpg')

In [ ]:
print('average check is', round(df.loc[df.event_type == 'purchase'].groupby('user_session')['price'].sum().median()), '$')

# Cumulative LTV by cohorts

In [ ]:
df['N_week'] = np.ceil(((df['event_time'] - df['start_date']).dt.total_seconds()) / (60 * 60 * 24 * 7))
df.loc[df.N_week == 0, 'N_week'] = 1
df.N_week = df.N_week.astype('int64')

In [ ]:
purchases = df.loc[df.event_type == 'purchase']
CUM_LTV = pd.DataFrame()
for week in df.start_week.unique():
    weakly_amount_by_cohort = purchases.loc[purchases.start_week == week].groupby('N_week')['price'].sum()
    users_by_cohort = df.loc[df.start_week == week, 'user_id'].nunique()
    cum_ltv = np.transpose((np.cumsum(weakly_amount_by_cohort) / users_by_cohort).reset_index())
    CUM_LTV = CUM_LTV.append(cum_ltv.iloc[1:]).rename(index = {'price': week})

In [ ]:
plt.figure(figsize = (16, 9))
j = sns.heatmap(CUM_LTV, annot = True, vmin = 0.0, vmax = 10.0, cmap="YlGnBu")

plt.title('Cumulative LTV by cohorts', fontsize = 20)
plt.ylabel('start week', fontsize = 17)
plt.xlabel('')

j.set_yticklabels(purchases.start_week.unique() ,fontsize = 13)
j.set_xticklabels(purchases.N_week.unique(),fontsize = 15)

plt.show()

In [ ]:
paying_share = round(df_purchases.groupby('start_week')['user_id'].nunique() / df.groupby('start_week')['user_id'].nunique() * 100, 2)

In [ ]:
plt.figure(figsize = (5, 12))
p = sns.barplot(x = paying_share.values, y = paying_share.index, color = '#654321')

plt.title('Paying share by cohorts', fontsize = 20)
plt.ylabel('start week', fontsize = 17)
plt.xlabel('')

p.set_yticklabels(purchases.start_week.unique() ,fontsize = 13)
p.set_xticklabels([str(x) + '%' for x in p.get_xticks()],fontsize = 15)

plt.show()